In [1]:
import pandas as pd
from zone_recovery.zone import *

In [15]:
df  = pd.read_csv('processed_data/BNBUSTD_2020_Processed.csv')

In [3]:
def makeDecision(prev_price, current_price, zone: Zone , state : str):

    if (prev_price > zone.up and current_price < zone.up or current_price == zone.up) and state == 'w8 4 Buy':
        return 'Open Buy', 'w8 4 Sell'

    if (prev_price < zone.down and current_price > zone.down or current_price == zone.down) and state == 'w8 4 Sell':
        return 'Open Sell', 'w8 4 Buy'

    if (prev_price < zone.above_target and current_price > zone.above_target or current_price == zone.above_target) and state == 'w8 4 Sell':
        return 'Close Buy', 'w8 4 Buy'

    if (prev_price > zone.below_target and current_price < zone.below_target or current_price == zone.below_target) and state == 'w8 4 Buy':
        return 'Close Sell', 'w8 4 Buy'

    return 'Steady', state



In [4]:
def caculateNextTradeAmount(expected_profit,entry_amount,buy_hold,sell_hold,zone_width=1,distance=4):
    
    expected_profit = (expected_profit )/ 100
    current_option_hold,opposit_option_hold = min(buy_hold,sell_hold) , max(buy_hold,sell_hold)

    if buy_hold + sell_hold == 0:
        return entry_amount
   
    return (   (opposit_option_hold)* (zone_width/100 + distance/100) + expected_profit) / (distance/100) - current_option_hold

In [17]:
caculateNextTradeAmount(1,10,1,0,zone_width=1,distance=4)

1.5

In [18]:
(   (1)* (1/100 + 4/100) + 0.01) / (4/100) - 0

1.5

In [5]:
def simulation(df, entry_amount, limit=99, expected_profit=4, multiplier=2, money=100, zone_width=1, distance=4):

    buy_unit, sell_unit, buy_hold, sell_hold = 0, 0, 0, 0
    alpha = entry_amount
    first_money = money

    log = []
    for i, current in df.iterrows():
        if(m<)

        if buy_hold == 0 and sell_hold == 0:
            zone = Zone(current.Price, current.Price *
                        zone_width / 100, current.Price * distance / 100)
            state, action = 'w8 4 Sell', 'Open Buy'
            alpha = entry_amount
        else:
            action, state = makeDecision(
                prev_price, current.Price, zone, state)
        
            if action != 'Steady':
                alpha = caculateNextTradeAmount(expected_profit,entry_amount,buy_hold,sell_hold,zone_width,distance)

        if  alpha > money:
            action = 'Cut Loss'

        if action == 'Open Buy':
            buy_hold += alpha
            money -= alpha
            buy_unit += alpha / current.Price
           
        if action == 'Open Sell':
            sell_hold += alpha
            money -= alpha
            sell_unit += alpha / current.Price

        if action == 'Cut Loss' or action == 'Close Buy' or action == 'Close Sell':
            money += buy_unit * current.Price + 2*sell_hold - sell_unit * current.Price
            buy_unit = 0
            sell_unit = 0
            buy_hold, sell_hold = 0, 0
            alpha = entry_amount
            state = 'w8 4 Sell'

        if(action != 'Steady'):
            print(action, current.Price, money, buy_hold, sell_hold)

        log.append((action, money, buy_hold, sell_hold))
        prev_price = current.Price
    print(money)
    return log

In [53]:
x = simulation(df,expected_profit = 0,entry_amount = 10,money = 100)

Open Buy 6.1222 90 10 0
Open Sell 6.068 77.5 10 12.5
Close Sell 5.8126 100.02042006767164 0 0
Open Buy 5.8260000000000005 90.02042006767164 10 0
Close Buy 6.0608 100.4234410082827 0 0
Open Buy 6.0625 90.4234410082827 10 0
Open Sell 6.002999999999999 77.9234410082827 10 12.5
Close Sell 5.7535 100.43328218734952 0 0
Open Buy 5.7559 90.43328218734952 10 0
Close Buy 5.9862 100.83339337760647 0 0
Open Buy 5.977 90.83339337760647 10 0
Open Sell 5.9211 78.33339337760647 10 12.5
Open Buy 5.9751 72.70839337760647 15.625 12.5
Close Buy 6.2906 100.87503585202704 0 0
Open Buy 6.2603 90.87503585202704 10 0
Open Sell 6.1999 78.37503585202704 10 12.5
Open Buy 6.2534 72.75003585202704 15.625 12.5
Open Sell 6.2075 65.71878585202704 15.625 19.53125
Open Buy 6.2437 56.92972335202704 24.4140625 19.53125
Open Sell 6.1979999999999995 45.94339522702704 24.4140625 30.517578125
Open Buy 6.2561 32.21048507077704 38.14697265625 30.517578125
Close Buy 6.5249 100.93357989586492 0 0
Open Buy 6.5258 90.9335798958649

In [40]:
x = simulation_list(df,expected_profit = 0,entry_amount = 10,money = 100,leverage=100)

Open Buy 13.8477 90 [10.0] [] 100.0
Open Sell 13.710999999999999 77.5 [0.1283245593129223] [12.5] 90.12832455931292
Open Buy 13.8181 71.875 [7.862460914086746, 5.625] [2.735941944423992] 88.09840285851072
Open Sell 13.7123 64.84375 [0.22220296511331838, 1.3181488410129205] [12.381482021734058, 7.031250000000089] 85.79683382786038
Open Buy 13.8468 56.0546875 [9.935007257522912, 6.793304614961535, 8.7890625] [0.1194296550213636, 0.13449945486909698] 81.82599098237492
Open Sell 13.7198 45.068359375 [0.7638091524223256, 1.623454925062049, 0.7279152493717689] [11.697724454817227, 6.646672832056044, 10.986328125] 77.51426411372941
Close Sell 13.140999999999998 104.21190025317559 [] [] 104.21190025317559
Open Buy 13.07 94.21190025317559 [10.0] [] 104.21190025317559
Close Buy 13.5963 144.47968908255575 [] [] 144.47968908255575
Open Buy 13.5439 134.47968908255575 [10.0] [] 144.47968908255575
Close Buy 14.0862 184.5198547659999 [] [] 184.5198547659999
Open Buy 14.0865 174.5198547659999 [10.0] []

In [15]:
x = simulation_list(df,expected_profit = 0,entry_amount = 10,money = 100,leverage=1)

Open Buy 6.1222 90 [10.0] [] 100.0
Open Sell 6.068 77.5 [9.911469733102479] [12.5] 99.91146973310248
Close Sell 5.8126 100.02042006767164 [] [] 100.02042006767164
Open Buy 5.8260000000000005 90.02042006767164 [10.0] [] 100.02042006767164
Close Buy 6.0608 100.4234410082827 [] [] 100.4234410082827
Open Buy 6.0625 90.4234410082827 [10.0] [] 100.4234410082827
Open Sell 6.002999999999999 77.9234410082827 [9.90185567010309] [12.5] 100.32529667838578
Close Sell 5.7535 100.43328218734952 [] [] 100.43328218734952
Open Buy 5.7559 90.43328218734952 [10.0] [] 100.43328218734952
Close Buy 5.9862 100.83339337760647 [] [] 100.83339337760647
Open Buy 5.977 90.83339337760647 [10.0] [] 100.83339337760647
Open Sell 5.9211 78.33339337760647 [9.906474820143885] [12.5] 100.73986819775035
Open Buy 5.9751 72.70839337760647 [9.996821147732977, 5.625] [12.386000911992705] 100.71621543733215
Close Buy 6.2906 100.87503585202704 [] [] 100.87503585202704
Open Buy 6.2603 90.87503585202704 [10.0] [] 100.8750358520270

In [24]:
x2 = simulation_list(df,expected_profit = 0,entry_amount = 10,money = 100,leverage=1,distance = 3,zone_width= 2)

Open Buy 13.8477 90 [10.0] [] 100.0
Open Sell 13.5822 73.33333333333333 [9.808271409692583] [16.666666666666668] 99.80827140969258
Close Sell 13.140999999999998 100.03105740808584 [] [] 100.03105740808584
Open Buy 13.07 90.03105740808584 [10.0] [] 100.03105740808584
Close Buy 13.589 100.42814998650971 [] [] 100.42814998650971
Open Buy 13.5963 90.42814998650971 [10.0] [] 100.42814998650971
Close Buy 14.0069 100.73014391132749 [] [] 100.73014391132749
Open Buy 14.011 90.73014391132749 [10.0] [] 100.73014391132749
Close Buy 14.4326 101.03105034198911 [] [] 101.03105034198911
Open Buy 14.4256 91.03105034198911 [10.0] [] 101.03105034198911
Close Buy 14.862 101.33356808821804 [] [] 101.33356808821804
Open Buy 14.8103 91.33356808821804 [10.0] [] 101.33356808821804
Open Sell 14.5197 74.66690142155137 [9.80378520354078] [16.666666666666668] 101.13735329175881
Open Buy 14.7912 56.88912364377359 [9.987103569812902, 17.777777777777782] [16.35502110925157] 101.00902610061584
Close Buy 15.2836 101.3

In [22]:
len(x2)

1745

In [44]:
x = simulation_list(df,expected_profit = 0,entry_amount = 10,money = 100,leverage=100,limit = 4)

Open Buy 13.8477 90 [10.0] [] 100.0
Open Sell 13.710999999999999 77.5 [0.1283245593129223] [12.5] 90.12832455931292
Open Buy 13.8181 71.875 [7.862460914086746, 5.625] [2.735941944423992] 88.09840285851072
Open Sell 13.7123 64.84375 [0.22220296511331838, 1.3181488410129205] [12.381482021734058, 7.031250000000089] 85.79683382786038
Cut Loss 13.716 85.68759874586584 [] [] 85.68759874586584
Open Buy 13.7147 75.68759874586584 [10.0] [] 85.68759874586584
Open Sell 13.5822 63.18759874586584 [0.3388335144041932] [12.5] 76.02643226027004
Close Sell 13.0107 86.95222901238084 [] [] 86.95222901238084
Open Buy 12.9913 76.95222901238084 [10.0] [] 86.95222901238084
Close Buy 13.589 132.9599418663677 [] [] 132.9599418663677
Open Buy 13.5963 122.9599418663677 [10.0] [] 132.9599418663677
Open Sell 13.5 110.4599418663677 [2.9171907062951163] [12.5] 125.87713257266282
Open Buy 13.5954 104.8349418663677 [9.933805520619678, 5.625] [3.666666666666707] 124.06041405365409
Close Buy 14.1458 136.35146101747137 [

In [43]:
def simulation_list(df, entry_amount, limit=99, expected_profit=4, multiplier=2, money=100, zone_width=1, distance=4,leverage=1):

    buy_unit, sell_unit, buy_hold, sell_hold = list(), list(), list(), list()
    buy_current, sell_current = list(),list()
    alpha = entry_amount
    first_money = money
    count = 0

    log = []
    for i, current in df.iterrows():
        if(money + sum(buy_current+[0]) + sum(sell_current+[0]) ) < 0.3 * first_money:
            print(action, current.Price, money, buy_current, sell_current,money +sum(buy_current+[0]) + sum(sell_current+[0]))

            print("Broken")
            return
        if len(buy_hold) == 0  and len(sell_hold) == 0:
            zone = Zone(current.Price, current.Price *
                        zone_width / 100, current.Price * distance / 100)
            state, action = 'w8 4 Sell', 'Open Buy'
            alpha = entry_amount
        else:
            action, state = makeDecision(
                prev_price, current.Price, zone, state)
        
            if action != 'Steady':
                alpha = caculateNextTradeAmount(expected_profit,entry_amount,sum(buy_hold),sum(sell_hold),zone_width,distance)

        if  alpha > money or count >= limit:
            action = 'Cut Loss'

        if action == 'Open Buy':
            buy_hold.append(alpha)
            money -= alpha
            buy_unit.append(alpha / current.Price)
            count += 1
            # buy_price.append(current.Price)
           
        if action == 'Open Sell':
            sell_hold.append(alpha)
            money -= alpha
            sell_unit.append(alpha / current.Price)
            count += 1
            # sell_price.append(current.Price)
        
        buy_current = changeValue(buy_hold,buy_unit,current.Price,1,leverage)
        sell_current = changeValue(sell_hold,sell_unit,current.Price,-1,leverage)
        
        if action == 'Cut Loss' or action == 'Close Buy' or action == 'Close Sell':
            money += sum(buy_current+[0]) + sum(sell_current+[0])
            buy_unit = []
            sell_unit = []
            buy_hold, sell_hold = [], []
            buy_current, sell_current = list(),list()
            alpha = entry_amount
            state = 'w8 4 Sell'
            count  = 0
       
        
        if(action != 'Steady'):
            print(action, current.Price, money, buy_current, sell_current,money +sum(buy_current+[0]) + sum(sell_current+[0]))


            log.append((action, money, buy_current, sell_current))
        prev_price = current.Price
    print(money)
    return log

In [8]:
def changeValue(holds,units,current_Price,state,leverage):
    out = holds.copy()
    for i in range(len(holds)):
        out[i] += (units[i]*current_Price-holds[i] ) * state * leverage
    return out
      

In [81]:
sum([])

0